In [ ]:
class CMA_ES:
    def __init__(self, x0, sigma, maxfevals = 10000, popsize = None, weights = None):
        N = x0.shape[0]
        self.dimension = N
        self.chiN = N**0.5 * (1 - 1. / (4 * N) + 1. / (21 * N**2))
        self.lam = 4 + int(3 * np.log(N)) if not popsize else popsize
        print(f"Popsize: {self.lam}")
        self.mu = int(self.lam / 2)
        
        if weights:
            self.weights = weights
        else:
            self.weights = np.array([np.log(self.lam / 2 + 0.5) - np.log(i + 1) if i < self.mu else 0
                        for i in range(self.lam)])
            self.weights /= np.sum(self.weights)
        self.mueff = np.sum(self.weights)**2 / np.sum(self.weights**2)
        
        self.cc = (4 + self.mueff/N) / (N+4 + 2 * self.mueff/N)
        self.cs = (self.mueff + 2) / (N + self.mueff + 5)
        self.c1 = 2 / ((N + 1.3)**2 + self.mueff) 
        self.cmu = min([1 - self.c1, 2 * (self.mueff - 2 + 1/self.mueff) / ((N + 2)**2 + self.mueff)])
        self.damps = 2 * self.mueff/self.lam + 0.3 + self.cs

        self.xmean = x0[:]
        self.sigma = sigma
        self.pc = np.zeros(N) 
        self.ps =np.zeros(N) 
        self.lazy_gap_evals = 0.5 * N * self.lam * (self.c1 + self.cmu)**-1 / N**2
        self.maxfevals = maxfevals
        self.C = np.identity(N)
        self.counteval = 0 
        self.fitvals = []   
        self.best = (x0, None)
        self.condition_number = 1
        self.eigen_values = np.ones(N)
        self.eigen_vectors = np.identity(N)
        self.updated_eval = 0
        self.inv_sqrt = np.identity(N)

    def _update_eigensystem(self, current_eval, lazy_gap_evals):
#         if current_eval <= self.updated_eval + lazy_gap_evals:
#             return self
        self.eigen_values, self.eigen_vectors = np.linalg.eig(self.C)
        self.inv_sqrt = self.eigen_vectors @ np.diag(self.eigen_values**-0.5) @ self.eigen_vectors.T
        self.condition_number = self.eigen_values.max() / self.eigen_values.min()
         
    def sample(self):
        """Wylosuj próbkę nowych osobników"""
        # TODO
        y = self.xmean + self.sigma * np.dot(np.random.randn(self.lam, self.dimension), 
                                                  np.linalg.cholesky(self.C).T)
        return y
    
    def update(self, x, fitvals):
        """Zaktualizuj wartości uzyskanych parametrów"""
        self.counteval += fitvals.shape[0] # Zwiększamy licznik wykonań
        N = self.xmean.shape[0]
        x_old = self.xmean.copy()
        
        # Posortuj osobniki po wartości funkcji celu
        # TODD
        I = np.argsort(fitvals)
        x = x[I, :]
        
        #TODO Wartości funkcji celu dla posortowanych użytkowników
        self.fitvals = fitvals[I]
        self.best = (x[0], self.fitvals[0])        
        self.xmean = (self.weights @ x).ravel()
        
#         self.xmean = x_old  + (self.weights @ (x - x_old)).ravel()
        

        # Wektor przesunięcia średniej
        y = (x - x_old)/self.sigma
#         print(self.inv_sqrt, self.inv_sqrt.shape, y, y.shape)
#         W sumie nie używane
        z = (self.inv_sqrt @ y.T).T
        
        # Aktualizacja ścieżki ewolucji dla sigmy

        self.ps = (1-self.cs) * self.ps + np.sqrt(self.cs*(2-self.cs)*self.mueff) * (self.inv_sqrt @ \
                (self.xmean - x_old) / self.sigma)
 
        # Aktualizacja ścieżki ewolucji dla macierzy kowariancji
        self.pc = (1-self.cc) * self.pc + np.sqrt(self.cc*(2-self.cc)*self.mueff) * (self.xmean - x_old)/self.sigma
        
        #Aktualizacja macierzy kowariancji
        C_mu = np.sum(np.array([w * np.outer(yi, yi) for w, yi in zip(self.weights, y)]), axis=0)
        
        self.C = (1-self.c1-self.cmu*np.sum(self.weights))*self.C +\
                 self.c1 * self.pc.reshape(-1,1) @ self.pc.reshape(1,-1) +\
                 self.cmu * C_mu

        self.C = (self.C + self.C.T)/2.0 # Upewniamy się, że macierz jest symetryczna
        
        # Aktualizacja rozmiaru kroku
        
        self.sigma = self.sigma * np.exp(self.cmu/self.damps * (np.linalg.norm(self.ps) / self.chiN - 1))
        
#         print("\nxmean:", self.xmean, "\nC:", self.C, "\nsigma:", self.sigma)
#         ????????
        self._update_eigensystem(1.0, 0.0) 
    
    def terminate(self):
        """Zakończ algorytm"""
        if self.counteval <= 0:
            return False
        if self.counteval >= self.maxfevals:
            return True
        if self.condition_number > 1e13:
            return True
        if self.sigma * np.max(self.eigen_values)**0.5 < 1e-13:
            return True
        return False